In [1]:
import math
import numpy as np
from submodlib import FacilityLocationFunction, GraphCutFunction, DisparityMinFunction, DisparitySumFunction
import pickle
from itertools import permutations, combinations
from torchvision import datasets
import pandas as pd
import os

/raid/ganesh/namitha/miniconda3/envs/submodlib-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
dataset = "cifar10"

train_dataset = datasets.CIFAR10(root="../../data", train=True, download=True)
test_dataset = datasets.CIFAR10(root="../../data", train=False, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [19]:
# import sys
# sys.path.append('..')
# from utils.feature_extrater.cifar_feature_extractor import extract_features_threaded_worker, extract_features_resnet_threaded_cifar

In [20]:
# def generate_permutations(data):
#     """Generates all permutations of a list."""
#     for permutation in permutations(data):
#         yield permutation

# def get_all_ordered_pairs(original_list):
#     """
#     This function takes a list and returns all ordered pairs (lists of 2 elements)
#     formed from the original list.
#     """
#     for per in permutations(original_list, 2):
#         yield per

In [21]:
metric = "cosine"

In [2]:
which_exp = 2

In [3]:
import argparse
parser = argparse.ArgumentParser(description="Program to generate sequential subset of data",
                                 formatter_class=argparse.RawTextHelpFormatter)

parser.add_argument("--exp", type=int,
                    default=2,
                    help="""
                        Why you want to generate subset?
                        2: parallel_seq_data_gen
                        4: seq_data_gen
                        5: maa chudaye duniya wale
                    """,
                    required=True)

_StoreAction(option_strings=['--exp'], dest='exp', nargs=None, const=None, default=2, type=<class 'int'>, choices=None, required=True, help='Why you want to generate subset?                            2: paraller_seq_data_geb                            4: seq_data_gen                            5: maa chudaye duniya wale', metavar=None)

In [5]:
args = parser.parse_args()
which_exp = args.exp

usage: ipykernel_launcher.py [-h] --exp EXP
ipykernel_launcher.py: error: the following arguments are required: --exp


SystemExit: 2

/raid/ganesh/namitha/miniconda3/envs/submodlib-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:

which_exp = input("Why you want to generate subset?\
                   2: paraller_seq_data_geb\
                   4: seq_data_gen\
                   5: maa chudaye duniya wale")
which_exp = int(which_exp)

In [24]:
func_list = ["facility-location", "disparity-min",  "disparity-sum", "graph-cut"]

In [25]:
if which_exp==4:
    subset_fraction_size = [0.5, 0.6, 0.5]
elif which_exp==2:
    subset_fraction_size = [0.3, 0.5]
else:
    print("Error: Experiment not defined!")
    exit()

In [3]:
dir_ls = os.listdir(f"./permutation_subsets_{which_exp}")
for i, s in enumerate(dir_ls):
    dir_ls[i]=dir_ls[i].split(".")[0]

In [26]:
for order in permutations(func_list, which_exp):
    per_func_list = list(order)
    filename = ""

    for func in per_func_list:
        filename += func+"_"
    filename = filename[:-1]

    if filename in dir_ls:
        continue;

    print(filename)

    with open(f"../../../milo-base/cifar10/dataframe.pkl", "rb") as f:
        df = pickle.load(f)
        
    groups = df.groupby('Label')
    dataframes = [group for _, group in groups]

    for i, df in enumerate(dataframes):
        df["Features"] = df["Features"].to_numpy()
        df["Index"] = df["Index"].to_numpy()

    list_indexes = []

    if which_exp==2:
        sz = 2
    elif which_exp==4:
        sz = 3

    for idx in range(sz):
        fraction_size = subset_fraction_size[idx]
        final_indexes = []
        for i, df in enumerate(dataframes):
            features = df["Features"].to_numpy()
            indexes = df["Index"].to_numpy()

            print("start", features.shape, indexes.shape)

            if which_exp==4:
                # can choose a different strategy 
                func = per_func_list[idx+i%2]
            elif which_exp==2:
                func = per_func_list[idx]

            if func=="facility-location":
                obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
            elif func=="disparity-min":
                obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="disparity-sum":
                obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="graph-cut":
                obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=0.45)
            else:
                raise Exception("Sorry, no submodlib function defined")
            
            S = obj.maximize(int(fraction_size*features.shape[0]), optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=False, costs=None, costSensitiveGreedy=False)
            # S = obj.maximize(30-10*idx, optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
            S = list(map(lambda tuple: tuple[0], S))

            print(type(S))
            indexes = indexes[S]
            features = features[S]

            final_indexes.extend(list(indexes))

            _df = pd.DataFrame()
            _df["Features"] = features
            _df["Index"] = indexes

            dataframes[i] = _df

            print("end", features.shape[0], indexes.shape[0])

        list_indexes.append(final_indexes)
        
    with open(f"permutation_subsets_{which_exp}/{filename}.pkl", "wb") as f:
        pickle.dump(list_indexes, f)

start (5000,) (5000,) 30 0


In [17]:
# if which_exp==1:
#     subset_fraction_size = [0.5, 0.6, 0.5]

#     for order in generate_permutations(func_list):
#         per_func_list = list(order)
#         filename = ""
#         for func in per_func_list:
#             filename += func+"_"
#         filename = filename[:-1]

#         with open(f"../../../milo-base/cifar10/dataframe.pkl", "rb") as f:
#             df = pickle.load(f)
            
#         groups = df.groupby('Label')
#         dataframes = [group for _, group in groups]

#         for i, df in enumerate(dataframes):
#             df["Features"] = df["Features"].to_numpy()
#             df["Index"] = df["Index"].to_numpy()


#         list_indexes = []
#         for idx in range(len(func_list)-1):
#             fraction_size = subset_fraction_size[idx]
#             final_indexes = []
#             for i, df in enumerate(dataframes):
#                 features = df["Features"].to_numpy()
#                 indexes = df["Index"].to_numpy()

#                 print("start", features.shape, indexes.shape, 30-10*idx, idx)

#                 # can choose a different strategy 
#                 func = per_func_list[idx+i%2]

#                 if func=="facility-location":
#                     obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
#                 elif func=="disparity-min":
#                     obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
#                 elif func=="disparity-sum":
#                     obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
#                 elif func=="graph-cut":
#                     obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=0.45)
#                 else:
#                     raise Exception("Sorry, no submodlib function defined")
                
#                 S = obj.maximize(int(fraction_size*features.shape[0]), optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
#                 # S = obj.maximize(30-10*idx, optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
#                 S = list(map(lambda tuple: tuple[0], S))

#                 print(type(S))
#                 indexes = indexes[S]
#                 features = features[S]

#                 final_indexes.extend(list(indexes))

#                 _df = pd.DataFrame()
#                 _df["Features"] = features
#                 _df["Index"] = indexes

#                 dataframes[i] = _df

#                 print("end", features.shape[0], indexes.shape[0])

#             list_indexes.append(final_indexes)
#         print(list_indexes)
            
#         with open(f"permutation_subsets/{filename}.pkl", "wb") as f:
#             pickle.dump(list_indexes, f)

start (5000,) (5000,) 30 0
<class 'list'>
end 2500 2500
start (2500,) (2500,) 20 1


[||||||||||||||||||||]100% [Iteration 2500 of 2500]

## 2 seq

In [13]:
# for i in permutations(func_list):
#     print(i)

('facility-location', 'disparity-min', 'disparity-sum', 'graph-cut')
('facility-location', 'disparity-min', 'graph-cut', 'disparity-sum')
('facility-location', 'disparity-sum', 'disparity-min', 'graph-cut')
('facility-location', 'disparity-sum', 'graph-cut', 'disparity-min')
('facility-location', 'graph-cut', 'disparity-min', 'disparity-sum')
('facility-location', 'graph-cut', 'disparity-sum', 'disparity-min')
('disparity-min', 'facility-location', 'disparity-sum', 'graph-cut')
('disparity-min', 'facility-location', 'graph-cut', 'disparity-sum')
('disparity-min', 'disparity-sum', 'facility-location', 'graph-cut')
('disparity-min', 'disparity-sum', 'graph-cut', 'facility-location')
('disparity-min', 'graph-cut', 'facility-location', 'disparity-sum')
('disparity-min', 'graph-cut', 'disparity-sum', 'facility-location')
('disparity-sum', 'facility-location', 'disparity-min', 'graph-cut')
('disparity-sum', 'facility-location', 'graph-cut', 'disparity-min')
('disparity-sum', 'disparity-min',

In [ ]:
# if which_exp==2:
#     subset_fraction_size = [0.3, 0.5]

#     for order in get_all_ordered_pairs(func_list):
#         per_func_list = list(order)
#         filename = ""
#         for func in per_func_list:
#             filename += func+"_"
#         filename = filename[:-1]

#         with open(f"../../../milo-base/cifar10/dataframe.pkl", "rb") as f:
#             df = pickle.load(f)
            
#         groups = df.groupby('Label')
#         dataframes = [group for _, group in groups]

#         for i, df in enumerate(dataframes):
#             df["Features"] = df["Features"].to_numpy()
#             df["Index"] = df["Index"].to_numpy()


#         list_indexes = []
#         for idx in range(len(func_list)-1):
#             fraction_size = subset_fraction_size[idx]
#             final_indexes = []
#             for i, df in enumerate(dataframes):
#                 features = df["Features"].to_numpy()
#                 indexes = df["Index"].to_numpy()

#                 print("start", features.shape, indexes.shape, 30-10*idx, idx)

#                 # can choose a different strategy 
#                 func = func_list[idx+i%2]

#                 if func=="facility-location":
#                     obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
#                 elif func=="disparity-min":
#                     obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
#                 elif func=="disparity-sum":
#                     obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
#                 elif func=="graph-cut":
#                     obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=0.45)
#                 else:
#                     raise Exception("Sorry, no submodlib function defined")
                
#                 S = obj.maximize(int(fraction_size*features.shape[0]), optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
#                 # S = obj.maximize(30-10*idx, optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
#                 S = list(map(lambda tuple: tuple[0], S))

#                 print(type(S))
#                 indexes = indexes[S]
#                 features = features[S]

#                 final_indexes.extend(list(indexes))

#                 _df = pd.DataFrame()
#                 _df["Features"] = features
#                 _df["Index"] = indexes

#                 dataframes[i] = _df

#                 print("end", features.shape[0], indexes.shape[0])

#             list_indexes.append(final_indexes)
#         print(list_indexes)
            
#         with open(f"permutation_subsets/{filename}.pkl", "wb") as f:
#             pickle.dump(list_indexes, f)